In [1]:
import polars as pl
import pandas as pd

In [2]:
df = pd.read_csv("report_conclusions_raw.csv")

In [3]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40574 entries, 0 to 40573
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   40574 non-null  int64 
 1   WritingDate  40574 non-null  object
 2   Title        40574 non-null  object
 3   Conclusion   40574 non-null  object
 4   OrgNameDisc  40574 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB
None
   Unnamed: 0 WritingDate                                       Title  \
0           0  2025-05-12           汉朔科技(301275)：电子价签行业领跑者 助力泛零售数字化升级   
1           1  2025-05-11             科大智能(300222)：战略聚焦数字能源 拓展智能机器人应用   
2           2  2025-05-09  海晨股份(300873)：业内稀缺跨领域物流方案提供商 AMHS业务构筑第二成长曲线   
3           3  2025-05-08              川环科技(300547)：汽车管路龙头 进军AIDC液冷可期   
4           4  2025-05-09              金力永磁(300748)：秉技术优势 乘行业东风 迎跨越发展   

                                          Conclusion OrgNameDisc  
0  　　核心结论：我们预计：公司有望在2025 年-2027 年分别实现归母净利润8.

In [4]:
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

In [5]:
#load a pretrained perplextiy model
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
def calculate_perplexity(text, stride=512):
    encodings = tokenizer(text, return_tensors="pt")
    nlls = []
    input_ids = encodings.input_ids
    seq_len = input_ids.size(1)

    for i in range(0, seq_len, stride):
        begin_loc = max(i + stride - model.config.n_positions, 0)
        end_loc = min(i + stride, seq_len)
        trg_len = end_loc - i
        input_ids_chunk = input_ids[:, begin_loc:end_loc]
        target_ids = input_ids_chunk.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids_chunk, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

In [7]:
df_sample=df[:10]
df_sample

,Unnamed: 0,WritingDate,Title,Conclusion,OrgNameDisc
0,0,2025-05-12,汉朔科技(301275)：电子价签行业领跑者 助力泛零售数字化升级,核心结论：我们预计：公司有望在2025 年-2027 年分别实现归母净利润8.18亿元、...,西部证券
1,1,2025-05-11,科大智能(300222)：战略聚焦数字能源 拓展智能机器人应用,公司深耕电力能源领域，聚焦数字能源战略成效初显。公司成立于2002 年，是一家深耕电力能...,广发证券
2,2,2025-05-09,海晨股份(300873)：业内稀缺跨领域物流方案提供商 AMHS业务构筑第二成长曲线,业内稀缺一体化供应链解决方案领军企业，下游优质客户增厚公司业绩弹性。公司长期聚焦服务于消...,民生证券
3,3,2025-05-08,川环科技(300547)：汽车管路龙头 进军AIDC液冷可期,公司基本情况：公司成立于2002 年，自设立以来一直专注于车用胶管系列产品，主营业务包括...,天风证券
4,4,2025-05-09,金力永磁(300748)：秉技术优势 乘行业东风 迎跨越发展,金力永磁是全球领先的高性能稀土永磁材料龙头企业。纵向来看，公司产能从2021年的1.5 ...,东方证券
5,5,2025-05-06,华策影视(300133)：优质内容商业价值有望借力AI持续增强,持续夯实内容 发挥长剧优势布局短剧\r\n 2024 年公司主业电视剧制作、电视剧版权...,华鑫证券
6,6,2025-05-05,新莱应材(300260)：2024年泛半导体支撑营收微增 成立合资子公司布局液冷领域,核心观点\r\n 2024 年公司实现营收28.49 亿元，同比增加5.08%，泛半导...,中信建投证券
7,7,2025-04-30,雷迪克(300652)：精密制造基因赋能 人形机器人丝杠新锐,汽车轴承隐形冠军，拓展OEM 主业迎拐点\r\n 公司是国内汽车轴承核心供应商，主要产...,开源证券
8,8,2025-04-30,首都在线(300846)：坚定全球化布局 AI助力再次腾飞,坚定全球化布局，从IDC到计算云再到智算云\r\n 公司成立于2005 年，是国内较早...,长江证券
9,9,2025-04-27,鼎捷数智(300378)：工业互联网龙头厂商 打造国内首个制造业多智能体协同平台,投资要点\r\n 深耕行业40 余年，发展成为工业制造领域软件龙头供应商\r\n 公...,浙商证券


In [20]:
df_sample_2=df[:1].copy()

In [21]:
df_sample_2["Title_ppl"] = df_sample_2["Title"].apply(calculate_perplexity)
df_sample_2["Conclusion_ppl"] = df_sample_2["Conclusion"].apply(calculate_perplexity)

In [ ]:
print(df_sample[["Title_ppl", "Conclusion_ppl"]].head())

In [10]:
import requests

API_KEY = "c5f912e5ed72441f8128100392d118e4"
API_URL = "https://api.gptzero.me/v2/predict/text"

In [ ]:
detection_lenght=10
for start_info in range(detection_lenght):

    text_to_check=df['Conclusion'][start_info]
    print(text_to_check)

    payload = {
        "document": text_to_check,
        "multilingual": False
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "x-api-key": API_KEY
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    print(response.status_code)
    if response.status_code == 200:
        data = response.json()
        print("Full API Response:")
        print(data)
        
        # gpt_prob = data["data"].get("is_gpt_generated", None)
        # human_prob = data["data"].get("is_human_written", None)
        # sentences = data["data"].get("gpt_generated_sentences", [])

        # print("\n--- Detection Results ---")
        # print(f"AI Generated Probability: {gpt_prob}%")
        # print(f"Human Written Probability: {human_prob}")
        # print("Detected AI-generated sentences:")
        # for sent in sentences:
        #     print("-", sent)

    else:
        print(f"Error: {response.status_code} - {response.text}")

　　核心结论：我们预计：公司有望在2025 年-2027 年分别实现归母净利润8.18亿元、9.68 亿元、11.66 亿元，对应当前PE 为29.25x、24.72x、20.54x。
　　首次覆盖，给予“买入”评级。
　　公司作为全球领先的智能IoT 硬件和行业数字化解决方案提供商之一，主要服务于商超、3C、餐饮等泛零售行业。公司围绕电子价签（Electronic ShelfLabel，ESL）物联网系统构建了全面的软硬件技术体系，并进一步将业务拓展到配件及其他智能硬件以及软件、软件运营服务（SaaS）及技术服务，有效帮助泛零售行业客户在数字化转型的行业生态中实现新的发展。
　　下游需求持续增长，公司与大客户保持良好的商务合作关系：根据公司招股说明书，欧洲、美国、日本等国家和地区目前已有大量商超应用电子价签，全球市场渗透率约15%。其中，法国作为电子价签使用量相对较多的国家，渗透率已经达30%以上。公司制定了聚焦优质大客户的经营战略，通过与全球零售大客户的深度合作，在零售业数字化转型进程中占据一定的先发优势。公司与主要客户建立较为良好的商务合作关系，有助于公司保持订单节奏的稳定性，有望助力公司业务实现可持续性较强、确定性较高的增长。
　　公司自主研发关键通信协议HiLPC，软硬件协同优势明显：公司自主研发的HiLPC 协议广泛适用于各产品条线通信及数据传输，具有抗干扰和吞吐能力强、并发量大、低功耗等优势。公司依托自身在通信协议等领域的技术积累，以软硬件结合的方式，有效适应零售业客户对线下门店的运营需求，有望进一步扩大自身竞争优势，在零售业数字化转型的背景下持续扩大市场份额。
　　同时，公司作为电子价签行业头部企业，有望在人工智能时代为零售业的数字化转型提供创新路径，助力零售业加速实现智能化转型。
　　风险提示：下游需求不及预期；贸易风险加剧；公司新产品落地和商业化不及预期；宏观经济不及预期。
200
Full API Response:
{'version': '2025-05-14-multilingual', 'neatVersion': '3.2m', 'scanId': '09b7656e-4a17-4560-b5fc-e17e6f31b34b', 'documents': [{'paragraphs': [{'start_sentence_index':

200
Full API Response:
{'version': '2025-05-14-multilingual', 'neatVersion': '3.2m', 'scanId': '4fd112b2-6a3d-4a7f-b6ea-e019234cd920', 'documents': [{'paragraphs': [{'start_sentence_index': 0, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 1, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 2, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 3, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 4, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 5, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 6, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}], 'completely_generated_prob': 0.002627274494396746, 'average_generated_prob': 0, 'predicted_class': 'human', 'confidence_score': 0.99732907234

In [14]:
data.keys()

dict_keys(['version', 'neatVersion', 'scanId', 'documents', 'editorDocumentId'])

In [15]:
data

{'version': '2025-05-14-multilingual',
 'neatVersion': '3.2m',
 'scanId': '4fd112b2-6a3d-4a7f-b6ea-e019234cd920',
 'documents': [{'paragraphs': [{'start_sentence_index': 0,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 1,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 2,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 3,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 4,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 5,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 6,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542}],
   'completely_generated_prob': 0.002627274494396746,
   'average_g

In [16]:
{'paragraphs': 
    [{'start_sentence_index': 0, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542},
     {'start_sentence_index': 1, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542},
     {'start_sentence_index': 2, 'num_sentences': 1, 'completely_generated_prob': 0.8181818033057853},
     {'start_sentence_index': 3, 'num_sentences': 1, 'completely_generated_prob': 0.8181818033057853},
     {'start_sentence_index': 4, 'num_sentences': 1, 'completely_generated_prob': 0.8181818033057853},
     {'start_sentence_index': 5, 'num_sentences': 1, 'completely_generated_prob': 0.8181818033057853},
     {'start_sentence_index': 6, 'num_sentences': 1, 'completely_generated_prob': 0.8181818033057853}],
    'completely_generated_prob': 0.8930491209030151, 'average_generated_prob': 1, 'predicted_class': 'ai', 'confidence_score': 0.8930491209030151, 'confidence_category': 'medium', 'overall_burstiness': 0, 'writing_stats': {}, 'confidence_thresholds_raw': {'identity': {'ai': {'reject': 0.7, 'low': 0.8, 'medium': 0.92}, 'human': {'reject': 0.7, 'low': 0.82, 'medium': 0.92}, 'mixed': {'reject': 0.7, 'low': 0.8, 'medium': 0.88}}}, 'sentences': [{'generated_prob': 0.7000816464424133, 'sentence': '核⼼结论:我们预计:公司有望在2025 年-2027 年分别实现归⺟净利润8.18亿元、9.68 亿元、11.66 亿元,对应当前PE 为29.25x、24.72x、20.54x˳', 'perplexity': 0, 'class_probabilities': {'human': 0.2999183237552643, 'ai': 0.7000816464424133, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}, {'generated_prob': 0.6891748905181885, 'sentence': '⾸次覆盖,给予“买⼊”评级˳', 'perplexity': 0, 'class_probabilities': {'human': 0.3108251094818115, 'ai': 0.6891748905181885, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}, {'generated_prob': 0.8374205827713013, 'sentence': '公司作为全球领先的智能IoT 硬件和⾏业数字化解决⽅案提供商之ᅳ,主要服务于商超、3C、餐饮等泛零售⾏业˳公司围绕电⼦价签(Electronic ShelfLabel,ESL)物联⽹系统构建了全⾯的软硬件技术体系,并进ᅳ步将业务拓展到配件及其他智能硬件以及软件、软件运营服务(SaaS)及技术服务,有效帮助泛零售⾏业客户在数字化转型的⾏业⽣态中实现新的发展˳', 'perplexity': 0, 'class_probabilities': {'human': 0.16257937252521515, 'ai': 0.8374205827713013, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}, {'generated_prob': 0.9951419830322266, 'sentence': '下游需求持续增⻓,公司与⼤客户保持良好的商务合作关系:根据公司招股说明书,欧洲、美国、⽇本等国家和地区⽬前已有⼤量商超应⽤电⼦价签,全球市场渗透率约15%˳其中,法国作为电⼦价签使⽤量相对较多的国家,渗透率已经达30%以上˳公司制定了聚焦优质⼤客户的经营战略,通过与全球零售⼤客户的深度合作,在零售业数字化转型进程中占据ᅳ定的先发优势˳公司与主要客户建⽴较为良好的商务合作关系,有助于公司保持订单节奏的稳定性,有望助⼒公司业务实现可持续性较强、确定性较⾼的增⻓˳', 'perplexity': 0, 'class_probabilities': {'human': 0.004857965745031834, 'ai': 0.9951419830322266, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}, {'generated_prob': 0.891999363899231, 'sentence': '公司⾃主研发关键通信协议HiLPC,软硬件协同优势明显:公司⾃主研发的HiLPC 协议⼴泛适⽤于各产品条线通信及数据传输,具有抗⼲扰和吞吐能⼒强、并发量⼤、低功耗等优势˳公司依托⾃⾝在通信协议等领域的技术积累,以软硬件结合的⽅式,有效适应零售业客户对线下⻔店的运营需求,有望进ᅳ步扩⼤⾃⾝竞争优势,在零售业数字化转型的背景下持续扩⼤市场份额˳', 'perplexity': 0, 'class_probabilities': {'human': 0.10800065100193024, 'ai': 0.891999363899231, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}, {'generated_prob': 0.8443769216537476, 'sentence': '同时,公司作为电⼦价签⾏业头部企业,有望在⼈⼯智能时代为零售业的数字化转型提供创新路径,助⼒零售业加速实现智能化转型˳', 'perplexity': 0, 'class_probabilities': {'human': 0.15562307834625244, 'ai': 0.8443769216537476, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}, {'generated_prob': 0.8287656307220459, 'sentence': '⻛险提⽰:下游需求不及预期;贸易⻛险加剧;公司新产品落地和商业化不及预期;宏观经济不及预期˳', 'perplexity': 0, 'class_probabilities': {'human': 0.1712343543767929, 'ai': 0.8287656307220459, 'paraphrased': 0}, 'highlight_sentence_for_ai': True, 'special_highlight_type': None}], 'class_probabilities': {'human': 0.10533474385738373, 'ai': 0.8930491209030151, 'mixed': 0.0016160516534000635}, 'confidence_scores_raw': {'identity': {'ai': 0.8930491209030151, 'mixed': 0.0016160516534000635, 'human': 0.10533474385738373}}, 'subclass': {'ai': {'predicted_class': 'pure_ai', 'result_message': '', 'confidence_score': 1, 'confidence_category': 'high', 'class_probabilities': {'pure_ai': 1, 'ai_paraphrased': 0}, 'confidence_scores_raw': {'identity': {'pure_ai': 1, 'ai_paraphrased': 0}}, 'confidence_thresholds_raw': {'identity': {'ai_paraphrased': {'reject': 0.7, 'low': 0.8, 'medium': 0.92}, 'pure_ai': {'reject': 0.7, 'low': 0.8, 'medium': 0.92}}}}}, 'pageNumber': 0, 'neatVersion': '3.2m', 'version': '2025-05-14-multilingual', 'language': 'zh', 'result_message': 'Our detector is moderately confident that the text is written by AI.', 'result_sub_message': None, 'document_classification': 'AI_ONLY', 'inputText': '\u3000\u3000核心结论：我们预计：公司有望在2025 年-2027 年分别实现归母净利润8.18亿元、9.68 亿元、11.66 亿元，对应当前PE 为29.25x、24.72x、20.54x。\r\n\u3000\u3000首次覆盖，给予“买入”评级。\r\n\u3000\u3000公司作为全球领先的智能IoT 硬件和行业数字化解决方案提供商之一，主要服务于商超、3C、餐饮等泛零售行业。公司围绕电子价签（Electronic ShelfLabel，ESL）物联网系统构建了全面的软硬件技术体系，并进一步将业务拓展到配件及其他智能硬件以及软件、软件运营服务（SaaS）及技术服务，有效帮助泛零售行业客户在数字化转型的行业生态中实现新的发展。\r\n\u3000\u3000下游需求持续增长，公司与大客户保持良好的商务合作关系：根据公司招股说明书，欧洲、美国、日本等国家和地区目前已有大量商超应用电子价签，全球市场渗透率约15%。其中，法国作为电子价签使用量相对较多的国家，渗透率已经达30%以上。公司制定了聚焦优质大客户的经营战略，通过与全球零售大客户的深度合作，在零售业数字化转型进程中占据一定的先发优势。公司与主要客户建立较为良好的商务合作关系，有助于公司保持订单节奏的稳定性，有望助力公司业务实现可持续性较强、确定性较高的增长。\r\n\u3000\u3000公司自主研发关键通信协议HiLPC，软硬件协同优势明显：公司自主研发的HiLPC 协议广泛适用于各产品条线通信及数据传输，具有抗干扰和吞吐能力强、并发量大、低功耗等优势。公司依托自身在通信协议等领域的技术积累，以软硬件结合的方式，有效适应零售业客户对线下门店的运营需求，有望进一步扩大自身竞争优势，在零售业数字化转型的背景下持续扩大市场份额。\r\n\u3000\u3000同时，公司作为电子价签行业头部企业，有望在人工智能时代为零售业的数字化转型提供创新路径，助力零售业加速实现智能化转型。\r\n\u3000\u3000风险提示：下游需求不及预期；贸易风险加剧；公司新产品落地和商业化不及预期；宏观经济不及预期。',
 'document_id': 'cfeafe3c-7945-466d-8092-6a77c7d7816f'}

{'paragraphs': [{'start_sentence_index': 0,
   'num_sentences': 1,
   'completely_generated_prob': 0.11111110864197542},
  {'start_sentence_index': 1,
   'num_sentences': 1,
   'completely_generated_prob': 0.11111110864197542},
  {'start_sentence_index': 2,
   'num_sentences': 1,
   'completely_generated_prob': 0.8181818033057853},
  {'start_sentence_index': 3,
   'num_sentences': 1,
   'completely_generated_prob': 0.8181818033057853},
  {'start_sentence_index': 4,
   'num_sentences': 1,
   'completely_generated_prob': 0.8181818033057853},
  {'start_sentence_index': 5,
   'num_sentences': 1,
   'completely_generated_prob': 0.8181818033057853},
  {'start_sentence_index': 6,
   'num_sentences': 1,
   'completely_generated_prob': 0.8181818033057853}],
 'completely_generated_prob': 0.8930491209030151,
 'average_generated_prob': 1,
 'predicted_class': 'ai',
 'confidence_score': 0.8930491209030151,
 'confidence_category': 'medium',
 'overall_burstiness': 0,
 'writing_stats': {},
 'confidence

In [17]:
for item in data['documents']:
    print(item,sep='\n')

{'paragraphs': [{'start_sentence_index': 0, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 1, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 2, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 3, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 4, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 5, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}, {'start_sentence_index': 6, 'num_sentences': 1, 'completely_generated_prob': 0.11111110864197542}], 'completely_generated_prob': 0.002627274494396746, 'average_generated_prob': 0, 'predicted_class': 'human', 'confidence_score': 0.9973290723455988, 'confidence_category': 'high', 'overall_burstiness': 0, 'writing_stats': {}, 'confidence_thresholds_raw': {'identity': {'ai': {'reject': 0.7, 

In [18]:
#sample
{'version': '2025-05-14-multilingual',
 'neatVersion': '3.2m',
 'scanId': 'ec100663-4c8e-4072-8711-589384a1a78b',
 'documents': [{'paragraphs': [{'start_sentence_index': 0,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 1,
     'num_sentences': 1,
     'completely_generated_prob': 0.11111110864197542},
    {'start_sentence_index': 2,
     'num_sentences': 1,
     'completely_generated_prob': 0.8181818033057853},
    {'start_sentence_index': 3,
     'num_sentences': 1,
     'completely_generated_prob': 0.8181818033057853},
    {'start_sentence_index': 4,
     'num_sentences': 1,
     'completely_generated_prob': 0.8181818033057853},
    {'start_sentence_index': 5,
     'num_sentences': 1,
     'completely_generated_prob': 0.8181818033057853},
    {'start_sentence_index': 6,
     'num_sentences': 1,
     'completely_generated_prob': 0.8181818033057853}],
   'completely_generated_prob': 0.8930491209030151,
...
   'result_sub_message': None,
   'document_classification': 'AI_ONLY',
   'inputText': '\u3000\u3000核心结论：我们预计：公司有望在2025 年-2027 年分别实现归母净利润8.18亿元、9.68 亿元、11.66 亿元，对应当前PE 为29.25x、24.72x、20.54x。\r\n\u3000\u3000首次覆盖，给予“买入”评级。\r\n\u3000\u3000公司作为全球领先的智能IoT 硬件和行业数字化解决方案提供商之一，主要服务于商超、3C、餐饮等泛零售行业。公司围绕电子价签（Electronic ShelfLabel，ESL）物联网系统构建了全面的软硬件技术体系，并进一步将业务拓展到配件及其他智能硬件以及软件、软件运营服务（SaaS）及技术服务，有效帮助泛零售行业客户在数字化转型的行业生态中实现新的发展。\r\n\u3000\u3000下游需求持续增长，公司与大客户保持良好的商务合作关系：根据公司招股说明书，欧洲、美国、日本等国家和地区目前已有大量商超应用电子价签，全球市场渗透率约15%。其中，法国作为电子价签使用量相对较多的国家，渗透率已经达30%以上。公司制定了聚焦优质大客户的经营战略，通过与全球零售大客户的深度合作，在零售业数字化转型进程中占据一定的先发优势。公司与主要客户建立较为良好的商务合作关系，有助于公司保持订单节奏的稳定性，有望助力公司业务实现可持续性较强、确定性较高的增长。\r\n\u3000\u3000公司自主研发关键通信协议HiLPC，软硬件协同优势明显：公司自主研发的HiLPC 协议广泛适用于各产品条线通信及数据传输，具有抗干扰和吞吐能力强、并发量大、低功耗等优势。公司依托自身在通信协议等领域的技术积累，以软硬件结合的方式，有效适应零售业客户对线下门店的运营需求，有望进一步扩大自身竞争优势，在零售业数字化转型的背景下持续扩大市场份额。\r\n\u3000\u3000同时，公司作为电子价签行业头部企业，有望在人工智能时代为零售业的数字化转型提供创新路径，助力零售业加速实现智能化转型。\r\n\u3000\u3000风险提示：下游需求不及预期；贸易风险加剧；公司新产品落地和商业化不及预期；宏观经济不及预期。',
   'document_id': 'cfeafe3c-7945-466d-8092-6a77c7d7816f'}],
 'editorDocumentId': None}

SyntaxError: ':' expected after dictionary key (1233056451.py, line 27)